<a href="https://colab.research.google.com/github/Sujeet2003/Machine-Learning/blob/main/TTL_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manual Code for extracting content using Table of Content



In [2]:
import pandas as pd
!pip install PyPDF2 --quiet
! pip install PyMuPDF --quiet
! pip install pytesseract --quiet
! pip install pillow --quiet
!apt-get install tesseract-ocr --quiet
! pip install pdfplumber --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 55.0 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1b

In [3]:
! pip install pdfplumber --quiet

In [6]:
# Table of Content
import PyPDF2

def extract_text_from_pdf(pdf_path, num_pages=30):

  extracted_text = ""
  with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages_in_pdf = len(pdf_reader.pages)

    for page_num in range(min(num_pages, num_pages_in_pdf)):
      page = pdf_reader.pages[page_num]
      extracted_text += page.extract_text()

  return extracted_text


pdf_path = '/content/Technical Deliverable Specification_SnPS _SP2103303_V1 2 (1) (1).pdf'

extracted_text = extract_text_from_pdf(pdf_path)

In [7]:
# Getting table of contents

import re
import csv

def extract_table_of_contents(text):

    start_markers = ["TABLE OF CONTENTS", "table of contents"]
    toc_entries = []
    found_toc = False
    max_lines_per_page = 250
    max_pages = 2
    max_lines_to_check = max_lines_per_page * max_pages
    lines_checked = 0

    toc_pattern = re.compile(r'^(\d+(\.\d+)*)(\s+)(.+?)(\s+)(\d+)$')

    for line in text.split('\n'):

        if any(marker in line for marker in start_markers):
            found_toc = True

        if found_toc:

            match = toc_pattern.match(line.strip())
            if match:
                level = match.group(1).strip()
                topic = match.group(4).strip()
                page = match.group(6).strip()
                toc_entries.append((level, topic, page))

            lines_checked += 1

            if lines_checked >= max_lines_to_check:
                break

    return toc_entries

def save_to_csv(toc_entries, filename):
    """Saves the table of contents entries to a CSV file."""
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Level", "Topic", "Page"])
        writer.writerows(toc_entries)


toc_entries = extract_table_of_contents(extracted_text)

save_to_csv(toc_entries, 'table_of_contents.csv')

print("Table of contents saved to 'table_of_contents.csv'")

Table of contents saved to 'table_of_contents.csv'


In [8]:
toc = pd.read_csv('/content/table_of_contents.csv')

In [84]:
toc.sample(10)

,Level,Topic,Page
123,6.4.1,Support to MSPA Activities,361
15,5.1.6,Analysis and Implementation - Minor,36
137,6.6.1,Support to BITE Integration Activity,394
0,1,SUBCONTRACTED ACTIVITIES DESCRIPTION,10
55,5.5.4,3DX DELMIA - 3D Flow simulations and record an...,182
136,6.6,AIOEYH –Health Engineering,394
24,5.2.2,Model Development - Medium,55
38,5.3.3,sFMS DEFINED SCOPE Work Package,88
80,5.6.9,Customization – ATA 31 System Definition Activ...,100
112,6.2.3,Further Requirements for 6.2.1 and 6.2.2,355


In [10]:
pdf_path = "/content/Technical Deliverable Specification_SnPS _SP2103303_V1 2 (1) (1).pdf"

In [11]:
import pdfplumber

In [177]:
import pdfplumber
import re

def extract_text_from_pdf(pdf_path, start_page, end_page):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(start_page - 1, end_page):  # pdfplumber uses 0-based indexing
            # Extract text from the current page
            page = pdf.pages[page_number]
            page_text = page.extract_text()

            if page_text:
                # Clean up the text: remove newlines and extra spaces
                cleaned_text = re.sub(r'\s+', ' ', page_text)  # Replace multiple spaces/newlines with a single space
                cleaned_text = cleaned_text.strip()  # Remove leading and trailing spaces

                extracted_text += cleaned_text + " "  # Add a space between page texts to keep them separate

    return extracted_text.lower().strip()  # Convert to lowercase and trim the final output

In [151]:
def extract_text_from_pdf(pdf_path, start_page, end_page):
    extracted_text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(start_page - 1, end_page):
            page = pdf.pages[page_number]
            page_text = page.extract_text()

            if page_text:
                # Split the text into lines for processing
                lines = page_text.split('\n')

                # Filter out headers and footers based on known patterns
                filtered_lines = []
                for line in lines:
                    # Check for header and footer patterns
                    if not re.search(r'Technical Deliverable Description|© AIRBUS|Date: \d{2}/\d{2}/\d{4}|AIRBUS AMBER|Reference_TDS_SP2103303 Page \d+ of \d+', line):
                        filtered_lines.append(line)

                # Join the filtered lines back into a single string
                cleaned_text = ' '.join(filtered_lines)

                # Clean up spaces and add to extracted text
                extracted_text += re.sub(r'\s+', ' ', cleaned_text).strip() + " "

    return extracted_text.lower().strip()

In [179]:
# Extracting content for the passed titles/topics
def extract_content(extracted_text, current_level, current_topic, next_level, next_topic):

  # Find the position of the current topic
  start_index = extracted_text.find(f"{current_level} {current_topic}")

  if start_index == -1:
    return None  # Current topic not found

  end_index = extracted_text.find(f"{next_level} {next_topic}", start_index) if next_topic else len(extracted_text)

  if end_index == -1:
    end_index = len(extracted_text)  # If next topic is not found, go to the end

  return extracted_text[start_index:end_index].strip()

In [21]:
csv = pd.read_csv('/content/table_of_contents.csv')

In [175]:
csv.sample()

,Level,Topic,Page,Content
96,5.7.1,Support to package development- Flight Control...,330,5.7.1 support to package development- flight c...


In [180]:
rows_count = len(csv)
content_list = []

for i in range(rows_count):
  current_level = csv.loc[i, 'Level']
  current_topic = csv.loc[i, 'Topic'].lower()
  current_page = csv.loc[i, 'Page']

  if i < rows_count - 1:
    next_level = csv.loc[i + 1, 'Level']
    next_topic = csv.loc[i + 1, 'Topic'].lower()
    next_page = csv.loc[i + 1, 'Page']
  else:
    next_level = None
    next_topic = None
  if next_page and current_page:
    extracted_text = extract_text_from_pdf(pdf_path, current_page, next_page)
  else:
    extracted_text = extract_text_from_pdf(pdf_path, current_page, current_page)

  content = extract_content(extracted_text, current_level, current_topic, next_level, next_topic)
  content_list.append(content)

csv['Content'] = content_list

In [184]:
csv.sample(10)

,Level,Topic,Page,Content
45,5.4.2,Electrical-Insitu,116,5.4.2 electrical-insitu 5.4.2.1 ela delivery (...
54,5.5.3,3DX DELMIA-create static plant Layout- Exsitu,178,5.5.3 3dx delmia-create static plant layout- e...
138,6.6.2,Support for Filter File Generation Activity,400,6.6.2 support for filter file generation activ...
95,5.7,AIOEYC- Aircraft Controls,330,5.7 aioeyc- aircraft controls
10,5.1.1,Model Integration - Minor,14,5.1.1 model integration - minor nominated purc...
56,5.5.5,3DX DELMIA - Analyze and compare simulation sc...,186,5.5.5 3dx delmia - analyze and compare simulat...
22,5.2,AIOEYM - Systems Modeling,51,5.2 aioeym - systems modeling
115,6.2.6,"Schedule, plan, milestones: NA",355,"6.2.6 schedule, plan, milestones: na major dat..."
140,6.6.4,Support for Modelling in Health Engineering De...,407,6.6.4 support for modelling in health engineer...
18,5.1.9,Further Requirements for 5.1.1 to 5.1.8,50,5.1.9 further requirements for 5.1.1 to 5.1.8 ...


In [182]:
csv.loc[10, 'Content']

'5.1.1 model integration - minor nominated purchaser contacts work package focal point: pankaj bhutani, hog-aioeys, airbus india. pankaj.bhutani@airbus.com batch no. swu_01 description the supplier shall provide support to aioeys in order to develop the software integration minor work unit for platform application under the responsibility of airbus india simulation product engineer (spe). the following steps of activities need to be completed for the model integration minor work unit of the simulation application development as per the aim. ● scd update/verification for model to be integrated ● check availability of model and related inputs in the model repository ● collect the model inputs like micds, model library, data files, documentation etc. from model repository ● perform the micd diff analysis. ● perform the connections to be added/modified for the model to be integrated. ● perform the model integration. ● deliver the working simulation application into the filer. ● perform the

In [88]:
csv.loc[csv['Level'] == '5.1.2']['Topic']

,Topic
11,Model Integration - Medium


In [49]:
# Current Pointer
# current_level = csv.loc[10, 'Level'].strip()
current_level = '5.1.1'
# current_topic = csv.iloc[10]['Topic'].strip()
current_topic = "Model Integration - Minor"
# current_page = csv.iloc[10]['Page']
start_page = 14

# Next Pointer
# next_level = csv.iloc[11]['Level'].strip()
next_level = '5.1.2'
# next_topic = csv.iloc[11]['Topic'].strip()
next_topic = "Model Integration - Medium"
# next_page = csv.iloc[11]['Page']
end_page = 19
extracted_text = extract_text_from_pdf(pdf_path, start_page, end_page)


extract_content(extracted_text, current_level, current_topic, next_level, next_topic)

In [52]:
print(extracted_text)

Technical Deliverable Description Appendix F
5 SUBCONTRACTED ACTIVITIES DESCRIPTION (WORK UNIT)
5.1 AIOEYS - Simulation platforms & Frameworks
The sections 5.1.1 to 5.1.8 describes the Work units to be executed for platforms and frameworks activities
5.1.1 MODEL INTEGRATION - MINOR
Nominated Purchaser Contacts
Work Package Focal Point: Pankaj BHUTANI, HOG-AIOEYS, Airbus India. pankaj.bhutani@airbus.com
Batch No.
SWU_01
Description
The Supplier shall provide support to AIOEYS in order to develop the software integration Minor work unit for platform application under the responsibility of Airbus India Simulation
Product Engineer (SPE).
The following steps of activities need to be completed for the Model Integration Minor Work Unit of the Simulation application development as per the AIM.
● SCD Update/Verification for model to be integrated
● Check availability of model and related inputs in the model repository
● Collect the Model inputs like MICDs, Model library, data files, documentati

In [62]:
current_level, current_topic, next_level, next_topic

('5.1.1', 'Model Integration - Minor', '5.1.2', 'Model Integration - Medium')

In [171]:
current_value = f'{current_level} {current_topic}'
current_value = current_value.lower()

In [172]:
extracted_text.find(current_value)

940

# Use Table of Content

In [173]:
! pip install PyPDF2 --quiet

In [174]:
from PyPDF2 import PdfReader

In [ ]:
reader = PdfReader('/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf')

In [ ]:
text = ''
for i, page in enumerate(reader.pages):
  content = page.extract_text()
  if content:
    text += content
  else:
    print(f'Page {i} is empty')
  text += page.extract_text()

In [ ]:
! pip install fitz PyMuPDF --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade PyMuPDF --quiet
!pip install --force-reinstall PyMuPDF --quiet

In [ ]:
import fitz  # PyMuPDF
import re

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

def extract_toc(text):
    """Extract Table of Contents from the given text."""
    toc_pattern = r'(\d+)\s+(.+?)\s+(\d+)'  # Page number, topic, page number
    toc = re.findall(toc_pattern, text)
    return [(topic.strip(), int(page)) for page, topic, _ in toc]

def get_content_by_page(pdf_path, page_number):
    """Extract content from a specific page in the PDF."""
    with fitz.open(pdf_path) as pdf:
        if page_number < len(pdf):
            return pdf[page_number].get_text()
        else:
            return "Page number out of range."

def main(pdf_path):
    # Extract text from the PDF
    pdf_text = extract_text_from_pdf(pdf_path)

    # Extract the Table of Contents
    toc = extract_toc(pdf_text)

    # Print the TOC
    for topic, page in toc:
        print(f"Topic: {topic}, Page: {page}")

    # Example: Get content for each topic
    for topic, page in toc:
        content = get_content_by_page(pdf_path, page - 1)  # Convert to 0-based index
        print(f"\nContent for '{topic}':\n{content[:500]}...")  # Print first 500 characters

# Example usage
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf"  # Replace with your PDF file path
main(pdf_path)

Topic: A Internship –, Page: 18
Topic: -, Page: 2023
Topic: General Description, Page: 1
Topic: Purpose of Hackathon, Page: 2
Topic: Background, Page: 1
Topic: Challenges, Page: 2
Topic: Hackathon Day, Page: 1
Topic: Hackathon Day, Page: 2
Topic: Hackathon Day, Page: 1
Topic: 14, Page: 3
Topic: Hackathon Day, Page: 2
Topic: Hackathon Day, Page: 3
Topic: References, Page: 22
Topic: Key factors in a hackathon, Page: 1
Topic: Wireframes, Page: 2
Topic: Designs, Page: 3
Topic: Prototypes, Page: 4
Topic: Home Page, Page: 5
Topic: Chat-Bot, Page: 6
Topic: Medications, Page: 7
Topic: Emergency Contact, Page: 8
Topic: Caregiver, Page: 9
Topic: CHAPTER, Page: 1
Topic: CHAPTER, Page: 3
Topic: and lasted until, Page: 2023
Topic: October, Page: 4
Topic: October, Page: 5
Topic: CHAPTER, Page: 6
Topic: CHAPTER, Page: 9
Topic: Hackathon Day, Page: 1
Topic: Figure, Page: 10
Topic: 4.2 Hackathon Day, Page: 11
Topic: Figure, Page: 13
Topic: CHAPTER, Page: 14
Topic: Hackathon Day, Page: 1
Topic: 5.2 Hack

In [ ]:
import fitz  # PyMuPDF
import re
import csv

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    toc = []
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic based on TOC."""
    topic_content = []
    for i, (level, title, page) in enumerate(toc):
        if i + 1 < len(toc):
            next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
        else:
            next_page = None  # Last topic, extract till end of document

        content = get_content_by_page_range(pdf_path, page - 1, next_page)  # 0-based index
        topic_content.append((title, content.strip()))
    return topic_content

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Topic', 'Content'])
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract Table of Contents
    toc = extract_toc(pdf_path)

    # Print TOC for review
    print("Table of Contents:")
    for level, title, page in toc:
        indent = "    " * (level - 1)  # Indent based on TOC level
        print(f"{indent}{title} - Page {page}")

    # Match topics with their corresponding content
    topic_content = match_topic_with_content(toc, pdf_path)

    # Store the data in CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data stored in {output_file}.")

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Your uploaded PDF path
output_file = 'output_topics_content.csv'
main(pdf_path, output_file)


Table of Contents:
ORGANIZATION CERTIFICATE - Page 3
ACKNOWLEDGEMENT - Page 4
ABSTRACT - Page 5
LIST OF FIGURES - Page 7
    1.1 General Description - Page 8
    1.2 Purpose of Hackathon - Page 9
        October 25 - Page 10
        October 26 - Page 10
        October 27 - Page 11
        October 28 - Page 11
        October 29 - Page 11
        October 30 - Page 12
        October 31 - Page 12
    3.1 Background - Page 13
    3.2 Challenges - Page 13
    4.1 Hackathon Day 1 - Page 16
        Designing the wire-frame - Page 16
    4.2 Hackathon Day 2 - Page 18
        Prototyping in Figma  - Page 19
    5.1 Hackathon Day 2 - 3 - Page 21
        Header - Page 21
        Logo and Name: Located at the top left corner, dis - Page 21
        Footer - Page 21
    5.2 Hackathon Day 4 - Page 22
        Home page - Page 22
        Medications - Page 22
    5.3 Hackathon Day 5 - Page 24
    Emergency page - Page 24
    Caregiver Details Section: - Page 25
    Title: "Caregiver Details" - Page 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/output_topics_content.csv')

In [ ]:
df

,Topic,Content
0,ORGANIZATION CERTIFICATE,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,1.1 General Description,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,1.2 Purpose of Hackathon,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,October 25,NaN
7,October 26,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,October 27,NaN
9,October 28,NaN


In [ ]:
df.iloc[1]['Content']

'INTERNSHIP REPORT\nAY (2023-24)\nDepartment of CSE, HKBKCE\nIV\nACKNOWLEDGEMENT\nI would like to express my regards and acknowledgement to all who helped me in completing\nthis Internship successfully.\nFirst, I would take this opportunity to express my heartfelt gratitude to the personalities of\nHKBK College of Engineering, Mr. C M Ibrahim, Chairman, HKBKGI and Mr. C M Faiz,\nDirector, HKBKGI for providing facilities throughout the course.\nI express my sincere gratitude to Dr. Mohammed Riyaz Ahmed, Principal, HKBCE for his\nsupport and which inspired us towards the attainment of knowledge.\nI consider it as great privilege to convey my sincere regards to Dr. Smitha Kurian, Associate\nProfessor and HOD, Department of CSE, HKBKCE for her constant encouragement\nthroughout the course of the internship.\nI would specially like to thank my guide, Prof. Najmusher H, Assistant Professor,\nDepartment of CSE for her vigilant supervision and her constant encouragement. She spent\nher preciou

In [ ]:
import fitz  # PyMuPDF
import re
import csv

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:
            end_page = pdf.page_count  # Extract till the last page if end_page is None

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (uppercase, bold, etc.)."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic based on TOC."""
    topic_content = []
    for i, (level, title, page) in enumerate(toc):
        if i + 1 < len(toc):
            next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
        else:
            next_page = None  # Last topic, extract till end of document

        content = get_content_by_page_range(pdf_path, page - 1, next_page)  # 0-based index
        topic_content.append((level, title, content.strip()))
    return topic_content

def extract_content_by_headings(text):
    """Extract content based on detected headings if TOC is not available."""
    headings = extract_headings(text)
    extracted_data = []

    for i in range(len(headings) - 1):
        topic = headings[i][0]
        start_pos = headings[i][1]
        end_pos = headings[i + 1][1]
        content = text[start_pos:end_pos].strip()

        # Assuming subheadings are part of the topic, we add them here
        extracted_data.append((1, topic, content))

    return extracted_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Level', 'Topic/Subtopic', 'Content'])
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    # If TOC is available, use it to extract content
    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        topic_content = match_topic_with_content(toc, pdf_path)

    # If no TOC is available, extract content using headings
    else:
        print("No Table of Contents found, extracting content based on headings...")
        pdf_text = extract_text_from_pdf(pdf_path)
        topic_content = extract_content_by_headings(pdf_text)

    # Store the extracted content in a CSV
    store_in_csv(topic_content, output_file)
    print(f"Data stored in {output_file}.")

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Your uploaded PDF path
output_file = 'output_topics.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data stored in output_topics.csv.


In [ ]:
df = pd.read_csv('/content/output_topics.csv')

In [ ]:
df

,Level,Topic/Subtopic,Content
0,1,ORGANIZATION CERTIFICATE,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,1,ACKNOWLEDGEMENT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,1,ABSTRACT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,1,LIST OF FIGURES,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,2,1.1 General Description,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,2,1.2 Purpose of Hackathon,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,3,October 25,NaN
7,3,October 26,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,3,October 27,NaN
9,3,October 28,NaN


In [ ]:
import fitz  # PyMuPDF
import csv

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC."""
    topic_content = []
    current_main_topic = ""
    current_sub_topic_1 = ""
    current_sub_topic_2 = ""

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Add the extracted data in a hierarchical structure (tree-like)
            if level == 1:  # Main Topic
                current_main_topic = title
                current_sub_topic_1 = ""
                current_sub_topic_2 = ""
                topic_content.append([current_main_topic, "", "", content.strip()])
            elif level == 2:  # Sub-topic 1
                current_sub_topic_1 = title
                current_sub_topic_2 = ""
                topic_content.append([current_main_topic, current_sub_topic_1, "", content.strip()])
            elif level == 3:  # Sub-topic 2
                current_sub_topic_2 = title
                topic_content.append([current_main_topic, current_sub_topic_1, current_sub_topic_2, content.strip()])

    return topic_content

def extract_content_by_headings(pdf_path):
    """Extract content based on detected headings if TOC is not available."""
    pdf_text = extract_text_from_pdf(pdf_path)
    headings = extract_headings(pdf_text)
    extracted_data = []

    for i in range(len(headings) - 1):
        topic = headings[i][0]
        start_pos = headings[i][1]
        end_pos = headings[i + 1][1]
        content = pdf_text[start_pos:end_pos].strip()

        # Assuming all detected headings are main topics
        extracted_data.append([topic, "", "", content])

    return extracted_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Content'])
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found, extracting content based on headings...")
        # Extract content using headings
        topic_content = extract_content_by_headings(pdf_path)

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (all caps, bold, etc.) in the absence of TOC."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_content.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data successfully stored in output_content.csv.


In [ ]:
df = pd.read_csv('/content/output_content.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Content
0,ORGANIZATION CERTIFICATE,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,LIST OF FIGURES,1.1 General Description,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,LIST OF FIGURES,1.2 Purpose of Hackathon,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,LIST OF FIGURES,1.2 Purpose of Hackathon,October 25,NaN
7,LIST OF FIGURES,1.2 Purpose of Hackathon,October 26,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,LIST OF FIGURES,1.2 Purpose of Hackathon,October 27,NaN
9,LIST OF FIGURES,1.2 Purpose of Hackathon,October 28,NaN


In [ ]:
import fitz  # PyMuPDF
import csv
import re

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Create a row for the current topic, filling parent topics accordingly
            row = [""] * (level - 1) + [title] + [""] * (10 - level) + [content.strip()]

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def extract_content_by_headings(pdf_path):
    """Extract content based on detected headings if TOC is not available."""
    pdf_text = extract_text_from_pdf(pdf_path)
    headings = extract_headings(pdf_text)
    extracted_data = []

    for i in range(len(headings) - 1):
        topic = headings[i][0]
        start_pos = headings[i][1]
        end_pos = headings[i + 1][1]
        content = pdf_text[start_pos:end_pos].strip()

        # Assuming all detected headings are main topics
        extracted_data.append([topic, "", "", content])

    return extracted_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found, extracting content based on headings...")
        # Extract content using headings
        topic_content = extract_content_by_headings(pdf_path)

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (all caps, bold, etc.) in the absence of TOC."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_topics.csv'
main(pdf_path, output_file)

Table of Contents found, extracting content based on TOC...
Data successfully stored in output_topics.csv.


In [ ]:
df = pd.read_csv('/content/output_topics.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Sub-topic 3,Sub-topic 4,Sub-topic 5,Sub-topic 6,Sub-topic 7,Sub-topic 8,Sub-topic 9,Content
0,ORGANIZATION CERTIFICATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,NaN,1.1 General Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,NaN,1.2 Purpose of Hackathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,NaN,NaN,October 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,October 26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,NaN,NaN,October 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,October 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import fitz  # PyMuPDF
import csv
import re

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []
    parent_stack = []  # Stack to keep track of parent topics

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Update the stack based on the current level
            while len(parent_stack) >= level:  # Pop from stack if going to a higher level
                parent_stack.pop()
            parent_stack.append(title)  # Push current title onto the stack

            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Prepare the row based on the hierarchy level
            row = [""] * (level - 1) + [parent_stack[-1]] + [""] * (10 - level) + [content.strip()]

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def extract_content_by_headings(pdf_path):
    """Extract content based on detected headings if TOC is not available."""
    pdf_text = extract_text_from_pdf(pdf_path)
    headings = extract_headings(pdf_text)
    extracted_data = []

    for i in range(len(headings) - 1):
        topic = headings[i][0]
        start_pos = headings[i][1]
        end_pos = headings[i + 1][1]
        content = pdf_text[start_pos:end_pos].strip()

        # Assuming all detected headings are main topics
        extracted_data.append([topic, "", "", content])

    return extracted_data

def extract_figures_and_tables(pdf_path):
    """Scan the PDF for figures and tables, extracting the necessary data and corresponding images."""
    figures_data = []
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text = page.get_text("text")
            # Modify this regex based on the actual format of the figures/tables in your PDF
            figure_pattern = re.compile(r'Figure (\d+): (.+?)\s*\n')
            matches = figure_pattern.findall(text)

            # Extract images corresponding to the figures
            for match in matches:
                figure_number, description = match
                # Assuming figures are images on the same page, you can customize the extraction
                images = page.get_images(full=True)
                for img_index, img in enumerate(images):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    figures_data.append((figure_number, description, image_bytes))  # Collect figure data

    return figures_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found, extracting content based on headings...")
        # Extract content using headings
        topic_content = extract_content_by_headings(pdf_path)

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

    # Extract figures and tables data if needed
    figures_data = extract_figures_and_tables(pdf_path)
    print(f"Extracted {len(figures_data)} figures and tables.")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (all caps, bold, etc.) in the absence of TOC."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_topics_new.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data successfully stored in output_topics_new.csv.
Extracted 20 figures and tables.


In [ ]:
df = pd.read_csv('/content/output_topics_new.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Sub-topic 3,Sub-topic 4,Sub-topic 5,Sub-topic 6,Sub-topic 7,Sub-topic 8,Sub-topic 9,Content
0,ORGANIZATION CERTIFICATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,NaN,1.1 General Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,NaN,1.2 Purpose of Hackathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,NaN,NaN,October 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,October 26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,NaN,NaN,October 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,October 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import fitz  # PyMuPDF
import csv
import re

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
            # Optionally, extract images or other media here
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []
    parent_stack = []  # Stack to keep track of parent topics

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Update the stack based on the current level
            while len(parent_stack) >= level:  # Pop from stack if going to a higher level
                parent_stack.pop()
            parent_stack.append(title)  # Push current title onto the stack

            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Prepare a row with empty strings for each potential subtopic
            row = [""] * 10  # Assuming a maximum of 10 levels deep

            # Fill the row based on parent_stack
            for j in range(len(parent_stack)):
                if j < len(row):  # Avoid index error
                    row[j] = parent_stack[j]
            row[len(parent_stack)] = content.strip()  # Place content in the next column after the last topic

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def extract_figures_and_tables(pdf_path):
    """Scan the PDF for figures and tables, extracting the necessary data and corresponding images."""
    figures_data = []
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text = page.get_text("text")
            # Modify this regex based on the actual format of the figures/tables in your PDF
            figure_pattern = re.compile(r'Figure (\d+): (.+?)\s*\n')
            matches = figure_pattern.findall(text)

            # Extract images corresponding to the figures
            for match in matches:
                figure_number, description = match
                # Assuming figures are images on the same page, you can customize the extraction
                images = page.get_images(full=True)
                for img_index, img in enumerate(images):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    figures_data.append((figure_number, description, image_bytes))  # Collect figure data

    return figures_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found.")

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

    # Extract figures and tables data if needed
    figures_data = extract_figures_and_tables(pdf_path)
    print(f"Extracted {len(figures_data)} figures and tables.")

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_topics_new_1.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data successfully stored in output_topics_new_1.csv.
Extracted 20 figures and tables.


In [ ]:
df = pd.read_csv('/content/output_topics_new_1.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Sub-topic 3,Sub-topic 4,Sub-topic 5,Sub-topic 6,Sub-topic 7,Sub-topic 8,Sub-topic 9,Content
0,ORGANIZATION CERTIFICATE,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACKNOWLEDGEMENT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABSTRACT,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LIST OF FIGURES,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LIST OF FIGURES,1.1 General Description,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LIST OF FIGURES,1.2 Purpose of Hackathon,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LIST OF FIGURES,1.2 Purpose of Hackathon,October 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LIST OF FIGURES,1.2 Purpose of Hackathon,October 26,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LIST OF FIGURES,1.2 Purpose of Hackathon,October 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LIST OF FIGURES,1.2 Purpose of Hackathon,October 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.iloc[1]['Sub-topic 1']

'INTERNSHIP REPORT\nAY (2023-24)\nDepartment of CSE, HKBKCE\nIV\nACKNOWLEDGEMENT\nI would like to express my regards and acknowledgement to all who helped me in completing\nthis Internship successfully.\nFirst, I would take this opportunity to express my heartfelt gratitude to the personalities of\nHKBK College of Engineering, Mr. C M Ibrahim, Chairman, HKBKGI and Mr. C M Faiz,\nDirector, HKBKGI for providing facilities throughout the course.\nI express my sincere gratitude to Dr. Mohammed Riyaz Ahmed, Principal, HKBCE for his\nsupport and which inspired us towards the attainment of knowledge.\nI consider it as great privilege to convey my sincere regards to Dr. Smitha Kurian, Associate\nProfessor and HOD, Department of CSE, HKBKCE for her constant encouragement\nthroughout the course of the internship.\nI would specially like to thank my guide, Prof. Najmusher H, Assistant Professor,\nDepartment of CSE for her vigilant supervision and her constant encouragement. She spent\nher preciou

In [ ]:
! pip install pytesseract --quiet
!apt-get install tesseract-ocr --quiet

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,603 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~gi

In [ ]:
import fitz  # PyMuPDF
import csv
import re

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            content += pdf[page_num].get_text("text")
    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []
    parent_stack = []  # Stack to keep track of parent topics and sub-topics

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Update the stack based on the current level
            while len(parent_stack) >= level:  # Pop from stack if going to a higher level
                parent_stack.pop()
            parent_stack.append(title)  # Push current title onto the stack

            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Ensure the row has exactly 11 elements (10 for sub-topic levels and 1 for content)
            row = [""] * 11  # Prepare an empty row with 10 sub-topic levels and 1 content column

            for j in range(level):
                row[j] = parent_stack[j]  # Fill the row with topic and sub-topic hierarchy

            row[10] = content.strip()  # Add content to the last column

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def extract_content_by_headings(pdf_path):
    """Extract content based on detected headings if TOC is not available."""
    pdf_text = extract_text_from_pdf(pdf_path)
    headings = extract_headings(pdf_text)
    extracted_data = []

    for i in range(len(headings) - 1):
        topic = headings[i][0]
        start_pos = headings[i][1]
        end_pos = headings[i + 1][1]
        content = pdf_text[start_pos:end_pos].strip()

        # Assuming all detected headings are main topics
        extracted_data.append([topic, "", "", content])

    return extracted_data

def extract_figures_and_tables(pdf_path):
    """Scan the PDF for figures and tables, extracting the necessary data and corresponding images."""
    figures_data = []
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text = page.get_text("text")
            # Modify this regex based on the actual format of the figures/tables in your PDF
            figure_pattern = re.compile(r'Figure (\d+): (.+?)\s*\n')
            matches = figure_pattern.findall(text)

            # Extract images corresponding to the figures
            for match in matches:
                figure_number, description = match
                # Assuming figures are images on the same page, you can customize the extraction
                images = page.get_images(full=True)
                for img_index, img in enumerate(images):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    figures_data.append((figure_number, description, image_bytes))  # Collect figure data

    return figures_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found, extracting content based on headings...")
        # Extract content using headings
        topic_content = extract_content_by_headings(pdf_path)

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

    # Extract figures and tables data if needed
    figures_data = extract_figures_and_tables(pdf_path)
    print(f"Extracted {len(figures_data)} figures and tables.")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (all caps, bold, etc.) in the absence of TOC."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_topics_1.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data successfully stored in output_topics_1.csv.
Extracted 20 figures and tables.


In [ ]:
df = pd.read_csv('/content/output_topics_1.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Sub-topic 3,Sub-topic 4,Sub-topic 5,Sub-topic 6,Sub-topic 7,Sub-topic 8,Sub-topic 9,Content
0,ORGANIZATION CERTIFICATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,LIST OF FIGURES,1.1 General Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,LIST OF FIGURES,1.2 Purpose of Hackathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,LIST OF FIGURES,1.2 Purpose of Hackathon,October 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LIST OF FIGURES,1.2 Purpose of Hackathon,October 26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,LIST OF FIGURES,1.2 Purpose of Hackathon,October 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LIST OF FIGURES,1.2 Purpose of Hackathon,October 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Better than all in case of given `Table of content`

In [ ]:
import fitz  # PyMuPDF
import csv
import re
from pytesseract import image_to_string  # OCR for image-to-text extraction
from PIL import Image
import io

def extract_toc(pdf_path):
    """Extract Table of Contents using PyMuPDF's built-in TOC extraction method."""
    with fitz.open(pdf_path) as pdf:
        toc = pdf.get_toc()
    return toc

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF, including text from images."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            page = pdf[page_num]
            content += page.get_text("text")  # Extract the regular text

            # Extract and process images for OCR
            images = page.get_images(full=True)
            for img in images:
                xref = img[0]
                base_image = pdf.extract_image(xref)
                image_bytes = base_image["image"]
                image = Image.open(io.BytesIO(image_bytes))

                # Use OCR to extract text from the image
                ocr_text = image_to_string(image)

                if ocr_text.strip():  # Only add OCR text if something was recognized
                    content += f"\n[Image OCR Text]: {ocr_text}\n"

    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []
    parent_stack = []  # Stack to keep track of parent topics and sub-topics

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Update the stack based on the current level
            while len(parent_stack) >= level:  # Pop from stack if going to a higher level
                parent_stack.pop()
            parent_stack.append(title)  # Push current title onto the stack

            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range, including OCR from images
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Ensure the row has exactly 11 elements (10 for sub-topic levels and 1 for content)
            row = [""] * 11  # Prepare an empty row with 10 sub-topic levels and 1 content column

            for j in range(level):
                row[j] = parent_stack[j]  # Fill the row with topic and sub-topic hierarchy

            row[10] = content.strip()  # Add content to the last column

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def extract_figures_and_tables(pdf_path):
    """Scan the PDF for figures and tables, extracting the necessary data and corresponding images."""
    figures_data = []
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text = page.get_text("text")
            # Modify this regex based on the actual format of the figures/tables in your PDF
            figure_pattern = re.compile(r'Figure (\d+): (.+?)\s*\n')
            matches = figure_pattern.findall(text)

            # Extract images corresponding to the figures
            for match in matches:
                figure_number, description = match
                # Assuming figures are images on the same page, you can customize the extraction
                images = page.get_images(full=True)
                for img_index, img in enumerate(images):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    figures_data.append((figure_number, description, image_bytes))  # Collect figure data

    return figures_data

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract the Table of Contents (TOC)
    toc = extract_toc(pdf_path)

    if toc:
        print("Table of Contents found, extracting content based on TOC...")
        # Extract content based on TOC topics and subtopics
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents found, extracting content based on headings...")
        # Extract content using headings
        topic_content = extract_content_by_headings(pdf_path)

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

    # Extract figures and tables data if needed
    figures_data = extract_figures_and_tables(pdf_path)
    print(f"Extracted {len(figures_data)} figures and tables.")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_headings(text):
    """Extract headings and subheadings by identifying patterns (all caps, bold, etc.) in the absence of TOC."""
    heading_pattern = re.compile(r'([A-Z][A-Z ]{3,})')  # Simplistic heading pattern (all uppercase words)
    matches = heading_pattern.finditer(text)

    headings = []
    for match in matches:
        heading = match.group(0).strip()
        start_pos = match.start()
        headings.append((heading, start_pos))

    # Append the ending position of the last content
    if headings:
        headings.append(("End", len(text)))
    return headings

# Example usage
pdf_path = '/content/drive/MyDrive/Colab Notebooks/Internship_Report.pdf'  # Replace with your PDF file path
output_file = 'output_topics_new.csv'
main(pdf_path, output_file)


Table of Contents found, extracting content based on TOC...
Data successfully stored in output_topics_new.csv.
Extracted 20 figures and tables.


In [ ]:
df = pd.read_csv('/content/output_topics_new.csv')

In [ ]:
df

,Topic,Sub-topic 1,Sub-topic 2,Sub-topic 3,Sub-topic 4,Sub-topic 5,Sub-topic 6,Sub-topic 7,Sub-topic 8,Sub-topic 9,Content
0,ORGANIZATION CERTIFICATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
1,ACKNOWLEDGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
2,ABSTRACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
3,LIST OF FIGURES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
4,LIST OF FIGURES,1.1 General Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
5,LIST OF FIGURES,1.2 Purpose of Hackathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
6,LIST OF FIGURES,1.2 Purpose of Hackathon,October 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LIST OF FIGURES,1.2 Purpose of Hackathon,October 26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERNSHIP REPORT\nAY (2023-24)\nDepartment of...
8,LIST OF FIGURES,1.2 Purpose of Hackathon,October 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LIST OF FIGURES,1.2 Purpose of Hackathon,October 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.iloc[10]['Content']

'INTERNSHIP REPORT\nAY (2023-24)\nDepartment of CSE, HKBKCE\n4\nOctober 27\nThe focus shifted to CSS, where learners discovered how to style HTML elements to enhance\nthe visual appeal of web pages. Basic concepts such as selectors, properties, and values were\nintroduced, along with the box model, which is fundamental to understanding layout.\nParticipants learned to apply styles using class and ID selectors, as well as inline styles. The\nday covered styling text, backgrounds, borders, and adding spacing through margins and\npadding. Learners also explored the use of colors and gradients in CSS. Practical exercises\nincluded styling a basic web page, reinforcing the concepts learned.\nOctober 28\nAdvanced CSS techniques were examined, with a particular focus on layout strategies like\nflexbox and grid. Flexbox was introduced as a powerful tool for creating flexible and\nresponsive layouts, with an emphasis on understanding its various properties and how they\ninteract. Grid layout wa

# If not given table of content

In [ ]:
import fitz  # PyMuPDF
import csv
import re
from pytesseract import image_to_string  # OCR for image-to-text extraction
from PIL import Image
import io

def detect_table_of_contents(text):
    """
    Detect if a page contains a Table of Contents by analyzing common TOC patterns such as numbering,
    topic names, and page numbers.
    """
    toc_candidates = []
    lines = text.split("\n")

    # More comprehensive pattern for TOC detection
    toc_pattern = re.compile(r'^\s*(\d+(\.\d+)*\s+)+([^\d\n]+)\s+(\d+)\s*$')  # Match numbered lines with topic names

    for line in lines:
        match = toc_pattern.match(line)
        if match:
            numbering = match.group(1).strip()
            topic = match.group(3).strip()
            page_number = match.group(4).strip()
            level = len(numbering.split())
            toc_candidates.append((level, topic, int(page_number)))

    # Print the detected candidates for debugging
    if toc_candidates:
        print(f"Detected TOC entries: {toc_candidates}")

    return toc_candidates if toc_candidates else None

def extract_possible_toc(pdf_path):
    """
    Scan the document to detect a possible Table of Contents.
    """
    with fitz.open(pdf_path) as pdf:
        # Increase the number of pages to check for TOC
        max_pages_to_check = min(pdf.page_count, 20)  # Check up to 20 pages

        for page_num in range(max_pages_to_check):
            page_text = pdf[page_num].get_text("text").strip()  # Strip any leading/trailing whitespace
            possible_toc = detect_table_of_contents(page_text)
            if possible_toc:
                print(f"Possible Table of Contents detected on page {page_num + 1}")
                return possible_toc
    return None

def get_content_by_page_range(pdf_path, start_page, end_page):
    """Extract content from a range of pages in the PDF, including text from images."""
    content = ""
    with fitz.open(pdf_path) as pdf:
        if end_page is None:  # If end_page is not provided, read till the last page
            end_page = pdf.page_count

        for page_num in range(start_page, end_page):
            page = pdf[page_num]
            content += page.get_text("text")  # Extract the regular text

            # Extract and process images for OCR
            images = page.get_images(full=True)
            for img in images:
                xref = img[0]
                base_image = pdf.extract_image(xref)
                image_bytes = base_image["image"]
                image = Image.open(io.BytesIO(image_bytes))

                # Use OCR to extract text from the image
                ocr_text = image_to_string(image)

                if ocr_text.strip():  # Only add OCR text if something was recognized
                    content += f"\n[Image OCR Text]: {ocr_text}\n"

    return content

def match_topic_with_content(toc, pdf_path):
    """Extract content for each topic and subtopic based on TOC and organize in a hierarchical structure."""
    topic_content = []
    parent_stack = []  # Stack to keep track of parent topics and sub-topics

    with fitz.open(pdf_path) as pdf:
        for i, (level, title, page) in enumerate(toc):
            # Ensure the level is within bounds (maximum of 10 sub-topic levels)
            if level > 10:
                level = 10

            # Update the stack based on the current level
            while len(parent_stack) >= level:  # Pop from stack if going to a higher level
                parent_stack.pop()
            parent_stack.append(title)  # Push current title onto the stack

            # Determine the end page (next topic's page minus 1) for the current topic
            if i + 1 < len(toc):
                next_page = toc[i + 1][2] - 1  # Next topic's starting page - 1
            else:
                next_page = pdf.page_count  # Last topic, extract till the end of the document

            # Extract content from the given page range, including OCR from images
            content = get_content_by_page_range(pdf_path, page - 1, next_page)  # Convert to 0-based index

            # Ensure the row has exactly 11 elements (10 for sub-topic levels and 1 for content)
            row = [""] * 11  # Prepare an empty row with 10 sub-topic levels and 1 content column

            for j in range(min(level, len(parent_stack))):  # Prevent index errors by limiting to row size
                row[j] = parent_stack[j]  # Fill the row with topic and sub-topic hierarchy

            row[10] = content.strip()  # Add content to the last column

            # Append row to topic_content
            topic_content.append(row)

    return topic_content

def store_in_csv(data, output_file):
    """Store extracted data in CSV format."""
    header = ['Topic', 'Sub-topic 1', 'Sub-topic 2', 'Sub-topic 3', 'Sub-topic 4', 'Sub-topic 5', 'Sub-topic 6', 'Sub-topic 7', 'Sub-topic 8', 'Sub-topic 9', 'Content']

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in data:
            writer.writerow(row)

def main(pdf_path, output_file):
    # Extract a possible Table of Contents by analyzing patterns
    toc = extract_possible_toc(pdf_path)

    if toc:
        print("Table of Contents detected, extracting content based on TOC...")
        # Extract content based on detected TOC
        topic_content = match_topic_with_content(toc, pdf_path)
    else:
        print("No Table of Contents detected, extracting content based on headings...")
        # If no TOC is found, fall back to content extraction based on headings
        topic_content = []  # Placeholder; implement extract_content_by_headings as needed

    # Store the extracted content in a CSV format
    store_in_csv(topic_content, output_file)
    print(f"Data successfully stored in {output_file}.")

# Example usage
pdf_path = '/content/Internship_Report_without_toc.pdf'  # Replace with your PDF file path
output_file = 'output.csv'
main(pdf_path, output_file)

Detected TOC entries: [(1, 'Hackathon Day', 1)]
Possible Table of Contents detected on page 16
Table of Contents detected, extracting content based on TOC...
Data successfully stored in output.csv.


In [ ]:
df = pd.read_csv('output.csv')